In [8]:
#from pose_estimator_2d import openpose_estimator
from utils import smooth, vis, camera
from bvh_skeleton import openpose_skeleton, h36m_skeleton, cmu_skeleton

import cv2
import importlib
import numpy as np
import os
from pathlib import Path
from IPython.display import HTML

DEBUG:h5py._conv:Creating converter from 7 to 5
DEBUG:h5py._conv:Creating converter from 5 to 7
DEBUG:h5py._conv:Creating converter from 7 to 5
DEBUG:h5py._conv:Creating converter from 5 to 7
DEBUG:matplotlib:matplotlib data path: c:\Python\Lib\site-packages\matplotlib\mpl-data
DEBUG:matplotlib:CONFIGDIR=C:\Users\dodom\.matplotlib
DEBUG:matplotlib:interactive is False
DEBUG:matplotlib:platform is win32
DEBUG:matplotlib:CACHEDIR=C:\Users\dodom\.matplotlib
DEBUG:matplotlib.font_manager:Using fontManager instance from C:\Users\dodom\.matplotlib\fontlist-v390.json


## Estimate 2D pose from video

In [11]:
from mediapipe import solutions
from mediapipe.framework.formats import landmark_pb2
import numpy as np
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
import cv2
import time

# Function to draw landmarks on the image
def draw_landmarks_on_image(rgb_image, detection_result):
    pose_landmarks_list = detection_result.pose_landmarks
    annotated_image = np.copy(rgb_image)

    # Loop through the detected poses to visualize.
    for idx in range(len(pose_landmarks_list)):
        pose_landmarks = pose_landmarks_list[idx]

        # Draw the pose landmarks.
        pose_landmarks_proto = landmark_pb2.NormalizedLandmarkList()
        pose_landmarks_proto.landmark.extend([
            landmark_pb2.NormalizedLandmark(x=landmark.x, y=landmark.y, z=landmark.z) for landmark in pose_landmarks
        ])
        solutions.drawing_utils.draw_landmarks(
            annotated_image,
            pose_landmarks_proto,
            solutions.pose.POSE_CONNECTIONS,
            solutions.drawing_styles.get_default_pose_landmarks_style())

        # Add numbers beside each landmark
        for i, landmark in enumerate(pose_landmarks):
            x = int(landmark.x * annotated_image.shape[1])
            y = int(landmark.y * annotated_image.shape[0])
            cv2.putText(annotated_image, str(i), (x, y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1, cv2.LINE_AA)

    return annotated_image

DEBUG:tensorflow:Falling back to TensorFlow client; we recommended you install the Cloud TPU client directly with pip install cloud-tpu-client.
DEBUG:jax._src.path:etils.epath was not found. Using pathlib for file I/O.


In [ ]:
# Initialize the MediaPipe Pose detector
base_options = python.BaseOptions(model_asset_path='pose_landmarker.task')
options = vision.PoseLandmarkerOptions(
    base_options=base_options,
    output_segmentation_masks=True)
detector = vision.PoseLandmarker.create_from_options(options)

# Open the video file
cap = cv2.VideoCapture('miscs/cxk.mp4')

# Define the complete MediaPipe to OpenPose joint mapping (33 to 25)
mediapipe_to_openpose = {
    0: 0,   # Nose
    13: 5,  # Left Shoulder
    14: 2,  # Right Shoulder
    15: 6,  # Left Elbow
    16: 3,  # Right Elbow
    17: 7,  # Left Wrist
    18: 4,  # Right Wrist
    23: 12, # Left Hip
    24: 9,  # Right Hip
    25: 13, # Left Knee
    26: 10, # Right Knee
    27: 14, # Left Ankle
    28: 11, # Right Ankle
    31: 19, # Left Big Toe
    32: 22, # Right Big Toe
    29: 20, # Left Small Toe
    30: 21, # Left Heel
    28: 23, # Right Small Toe
    27: 24, # Right Heel
    1: 16,  # Left Eye
    2: 15,  # Right Eye
    3: 18,  # Left Ear
    4: 17,  # Right Ear
}

# Add interpolated joints
def interpolate_joint(lm1, lm2):
    """Interpolate between two landmarks."""
    return [(lm1.x + lm2.x) / 2, (lm1.y + lm2.y) / 2, (lm1.z + lm2.z) / 2]

# Process video frames
keypoints_list = []

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    img_height, img_width = frame.shape[:2]
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=rgb_frame)
    detection_result = detector.detect(mp_image)

    keypoints = np.zeros((25, 3))  # OpenPose format: 25 joints

    if detection_result.pose_landmarks:
        landmarks = detection_result.pose_landmarks[0]

        # Map MediaPipe landmarks to OpenPose
        for mp_idx, openpose_idx in mediapipe_to_openpose.items():
            if mp_idx < len(landmarks):
                landmark = landmarks[mp_idx]
                keypoints[openpose_idx, 0] = landmark.x * img_width
                keypoints[openpose_idx, 1] = landmark.y * img_height
                keypoints[openpose_idx, 2] = 1.0  # Confidence

        # Add interpolated joints
        # Neck (midpoint between left and right shoulders)
        neck = interpolate_joint(landmarks[13], landmarks[14])
        keypoints[1] = [neck[0] * img_width, neck[1] * img_height, 1.0]

        # MidHip (midpoint between left and right hips)
        mid_hip = interpolate_joint(landmarks[23], landmarks[24])
        keypoints[8] = [mid_hip[0] * img_width, mid_hip[1] * img_height, 1.0]

    keypoints_list.append(keypoints)
    annotated_image = draw_landmarks_on_image(rgb_frame, detection_result)
    cv2.imshow('Pose Detection', cv2.cvtColor(annotated_image, cv2.COLOR_RGB2BGR))

    # Resize the visualization window
    cv2.namedWindow('Pose Detection', cv2.WINDOW_NORMAL)
    cv2.resizeWindow('Pose Detection', 1500, 750)

    if cv2.waitKey(5) & 0xFF == 27:  # Press 'Esc' to exit
        break

cap.release()
cv2.destroyAllWindows()

print(f"2D pose estimation completed")


c:\Python\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


2D pose estimation completed


In [ ]:
print(keypoints_list)

## Process 2D pose

In [35]:
# save 2d pose result
pose2d = np.stack(keypoints_list)[:, :, :2]
pose2d_file = Path('2d_pose.npy')
np.save(pose2d_file, pose2d)

In [14]:
# Load the data from the .npy file
data = np.load('miscs/cxk_cache/2d_pose.npy')

# Print the shape of the data
print(data)

[[[ 777.48505  415.6107 ]
  [ 818.7475   459.73688]
  [ 745.1361   480.1361 ]
  ...
  [ 836.4469  1057.2075 ]
  [   0.         0.     ]
  [ 812.8102  1010.0808 ]]

 [[ 774.5472   409.54886]
  [ 818.75323  453.66135]
  [ 745.16364  459.80325]
  ...
  [ 833.38257 1054.2017 ]
  [ 806.92896 1048.4553 ]
  [ 812.9307  1004.1141 ]]

 [[ 774.63214  391.97034]
  [ 815.8464   436.0819 ]
  [ 742.2773   453.6574 ]
  ...
  [ 824.64166 1051.2272 ]
  [ 795.23975 1048.3055 ]
  [ 818.77026 1007.0596 ]]

 ...

 [[ 924.6487   403.747  ]
  [ 912.872    462.6452 ]
  [ 836.3551   483.2289 ]
  ...
  [ 974.73535 1074.8062 ]
  [ 954.14636 1074.8384 ]
  [ 957.06793 1074.8254 ]]

 [[ 933.45807  400.83636]
  [ 918.81116  459.80594]
  [ 836.45636  483.1317 ]
  ...
  [ 971.799   1074.8057 ]
  [ 954.0584  1074.8337 ]
  [ 957.14044 1060.0922 ]]

 [[ 954.06696  397.87613]
  [ 924.65173  462.59933]
  [ 839.3046   483.21722]
  ...
  [ 968.8072  1074.8271 ]
  [ 951.15234 1074.8523 ]
  [ 965.8592  1074.8138 ]]]


In [26]:
# Load the data from the .npy file
data = np.load('2d_pose.npy')

# Print the shape of the data
print(data)

[[[ 765.38497925  429.10190105]
  [ 844.97091293  562.24341989]
  [ 699.09885406  547.54277945]
  ...
  [ 824.2971611  1076.93120956]
  [ 816.46642685 1002.90159702]
  [ 942.26200104  944.5522213 ]]

 [[ 771.18896484  415.78434706]
  [ 840.21746635  547.24051595]
  [ 687.75690079  531.54061317]
  ...
  [ 822.35578537 1072.17481613]
  [ 816.59591675  998.9479351 ]
  [ 942.11483002  938.76907825]]

 [[ 764.46292877  400.8792901 ]
  [ 838.46334457  534.97050405]
  [ 678.3385849   519.82298613]
  ...
  [ 820.82662582 1071.71892643]
  [ 816.86336517  998.4338522 ]
  [ 941.28284454  924.51676369]]

 ...

 [[ 919.19179916  409.35454488]
  [ 939.5735836   637.85046101]
  [ 849.40126419  644.06329393]
  ...
  [ 982.46898651 1086.068573  ]
  [ 960.3560257  1033.03891897]
  [ 913.7951088   951.99985743]]

 [[ 934.48408127  407.36937761]
  [ 944.1975975   642.28283286]
  [ 850.44467926  647.26907015]
  ...
  [ 981.9644165  1095.34189224]
  [ 963.8344574  1036.72678471]
  [ 950.94509125  990.938515

## Visualize 2D pose

In [17]:
from pathlib import Path

# Set the path to save the visualized images
vis_result_dir = Path('2d_pose_vis')

# Ensure the directory exists
vis_result_dir.mkdir(parents=True, exist_ok=True)

cap = cv2.VideoCapture(str('trial.mp4'))
op_skel = openpose_skeleton.OpenPoseSkeleton()

for i, keypoints in enumerate(keypoints_list):
    ret, frame = cap.read()
    if not ret:
        break
    
    # keypoint whose detect confidence under kp_thresh will not be visualized
    output_file = vis_result_dir / f'{i:04d}.png'  # Correct path joining
    vis.vis_2d_keypoints(
        keypoints=keypoints,
        img=frame,
        skeleton=op_skel,
        kp_thresh=0.4,
        output_file=str(output_file)  # Convert to string if necessary
    )

cap.release()


## Initialize 3D pose estimator (Still working on it)

In [ ]:
import torch
import pathlib
import importlib
from pose_estimator_3d import estimator_3d 

# Fix pathlib.PosixPath issue on Windows
temp = pathlib.PosixPath
pathlib.PosixPath = pathlib.WindowsPath

# Reload the `estimator_3d` module to ensure it's fresh
importlib.reload(estimator_3d)

# Initialize Estimator3D with the configuration and checkpoint
e3d = estimator_3d.Estimator3D(
    config_file='models/video_pose.yaml',
    checkpoint_file='models/best_58.58.pth'
)

# Revert pathlib changes (optional)
pathlib.PosixPath = temp

# (Optional) Test the e3d object to ensure it works
print("e3d initialized:", e3d)


=> Read 3D estimator config from C:\Users\dodom\OneDrive - Misr International University\Desktop\College\college\Graduation Project\video2bvh-master\video2bvh-master\models\linear_model.yaml.
{'DATASET': {'CAM_PARAMS': '/home/kevin/public98/3dpose/Dataset/h36m/cameras.h5',
             'IMAGE_HEIGHT': 1002,
             'IMAGE_WIDTH': 1000,
             'INPUT_LEFT_JOINTS': [5, 6, 7, 12, 13, 14, 16, 18, 19, 20, 21],
             'INPUT_RIGHT_JOINTS': [2, 3, 4, 9, 10, 11, 15, 17, 22, 23, 24],
             'INPUT_ROOT': '/home/kevin/HDD/h36m_dataset/2D_openpose',
             'IN_CHANNEL': 2,
             'IN_JOINT': 25,
             'NAME': 'h36m',
             'OUTPUT_LEFT_JOINTS': [4, 5, 6, 11, 12, 13],
             'OUTPUT_RIGHT_JOINTS': [1, 2, 3, 14, 15, 16],
             'OUT_CHANNEL': 3,
             'OUT_JOINT': 17,
             'SEQ_LEN': 1,
             'TARGET_ROOT': '/home/kevin/HDD/h36m_dataset/3D_gt',
             'TEST_FLIP': True,
             'TRAIN_FLIP': True},
 'MODEL

c:\Users\dodom\OneDrive - Misr International University\Desktop\College\college\Graduation Project\video2bvh-master\video2bvh-master\pose_estimator_3d\model\factory.py:36: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any is

## Estimate 3D pose from 2D pose (still working on it)

In [40]:
pose2d = np.load('2d_pose.npy')
pose3d = e3d.estimate(pose2d, image_width=img_width, image_height=img_height)

=> Begin to estimate 3D poses.
161 / 161


In [20]:
# Load the data from the .npy file
data3d = np.load('miscs/cxk_cache/3d_pose.npy')

# Print the shape of the data
print(data3d)

[[[   0.            0.          922.10063329]
  [-128.08416251  -19.41330654  922.77364049]
  [ -37.19650811 -271.5757016   637.30918071]
  ...
  [-180.42006445 -255.89631158 1230.35682424]
  [-347.33498528 -146.71871621 1050.82630431]
  [-446.6621912  -251.62536351  987.74473772]]

 [[   0.            0.          922.10063329]
  [-128.93817239  -19.08141257  922.4809365 ]
  [ -40.23146449 -271.72872311  636.97512645]
  ...
  [-184.53306805 -256.12075935 1231.82107282]
  [-353.64487189 -147.78371802 1051.16152275]
  [-456.14752477 -257.54998149  991.01831833]]

 [[   0.            0.          922.10063329]
  [-127.93546685  -23.10065644  921.94295637]
  [ -35.9186382  -279.8481499   624.86144092]
  ...
  [-190.30998272 -260.88255489 1229.46203965]
  [-363.84663267 -151.20142688 1055.41885724]
  [-474.9994782  -258.39727163  995.44238242]]

 ...

 [[   0.            0.          922.10063329]
  [-107.03742218  -56.11416764  950.93654898]
  [  72.18954569 -279.61302081  678.32848957]
  ..

In [29]:
# Load the data from the .npy file
pose3d = np.load('3d_pose.npy')

# Print the shape of the data
print(pose3d)

[[[   0.            0.            0.        ]
  [-181.11601257   20.93011284  -98.03192902]
  [-160.63142395  392.71133423 -455.55709839]
  ...
  [-198.18856812 -334.14736938  -50.52825165]
  [-402.65106201 -122.14024353   -7.50772285]
  [-477.01257324  -92.36289978    9.52581024]]

 [[   0.            0.            0.        ]
  [-182.08384705   21.12501907  -98.29653168]
  [-162.62481689  394.34857178 -456.06637573]
  ...
  [-199.91113281 -335.76672363  -50.8671608 ]
  [-405.87310791 -123.07092285   -9.77371979]
  [-477.39004517  -92.79072571    5.78031588]]

 [[   0.            0.            0.        ]
  [-183.63911438   21.32544327  -98.27884674]
  [-164.32064819  396.27877808 -456.73031616]
  ...
  [-202.03178406 -338.15371704  -50.7162056 ]
  [-410.38470459 -126.33040619  -12.05461121]
  [-477.7696228   -94.51605225    1.70915389]]

 ...

 [[   0.            0.            0.        ]
  [-144.59503174   26.4815979  -115.93972015]
  [ -86.97019196  417.74621582 -431.42269897]
  ..

In [41]:
pose3d_world = pose3d  # Data is already in world coordinates

pose3d_file =  '3d_pose.npy'
np.save(pose3d_file, pose3d_world)

## Convert 3D pose to BVH

In [42]:
bvh_file = f'{"hellococo"}.bvh'
cmu_skel = cmu_skeleton.CMUSkeleton()
channels, header = cmu_skel.poses2bvh(pose3d_world, output_file=bvh_file)
print("Converted succesfully")

Converted succesfully


In [6]:
import os
import subprocess
import logging

logging.basicConfig(level=logging.DEBUG)

def open_blender_with_bvh(
    bvh_file,
    blender_executable=r"C:/Program Files/Blender Foundation/Blender 3.5/blender.exe"
):
    """
    Opens Blender, imports a BVH file, and ensures armature visibility.
    
    Parameters:
        bvh_file (str): Path to the BVH file to be imported.
        blender_executable (str): Path to the Blender executable.
    """
    # Check if the BVH file exists
    if not os.path.exists(bvh_file):
        raise FileNotFoundError(f"The BVH file '{bvh_file}' does not exist.")
    
    # Create a temporary Python script to run in Blender
    blender_script = f"""
import bpy
import os

# Path to the BVH file
bvh_file = r"{os.path.abspath(bvh_file)}"

def import_bvh(filepath, scale=1.0):
    # Ensure everything is deselected
    bpy.ops.object.select_all(action='DESELECT')

    # Import the BVH file
    bpy.ops.import_anim.bvh(
        filepath=filepath,
        axis_forward='-Z',   # Adjust if necessary
        axis_up='Y',         # Adjust if necessary
        filter_glob="*.bvh",
        target='ARMATURE',
        global_scale=scale,
        frame_start=1,
        use_fps_scale=True
    )

def set_armature_display():
    for obj in bpy.context.scene.objects:
        if obj.type == 'ARMATURE':
            bpy.context.view_layer.objects.active = obj  # Make it the active object
            obj.select_set(True)
            # Ensure it's visible in the viewport
            obj.hide_viewport = False
            obj.hide_set(False)
            # Change display mode for better visibility
            obj.data.display_type = 'STICK'  # Other options: 'ENVELOPE', 'OCTAHEDRAL', etc.

# Clear existing scene
bpy.ops.wm.read_factory_settings(use_empty=True)

# Import BVH file
import_bvh(bvh_file, scale=10.0)

# Adjust armature visibility and display settings
set_armature_display()

# Optionally save the Blender file
output_blend = os.path.splitext(bvh_file)[0] + ".blend"
bpy.ops.wm.save_as_mainfile(filepath=output_blend)

print("BVH file imported and processed successfully.")
    """
    
    # Write the Blender script to a temporary file
    script_file = "import_bvh_to_blender.py"
    with open(script_file, "w") as file:
        file.write(blender_script)
    
    # Run Blender with the temporary script
    try:
        subprocess.run([blender_executable, "--python", script_file], check=True)
    finally:
        # Clean up the temporary script
        if os.path.exists(script_file):
            os.remove(script_file)

# Example usage
bvh_file = "miscs/cxk_cache/cxk.bvh"  # Replace with the correct path to your BVH file
try:
    open_blender_with_bvh(bvh_file)
except Exception as e:
    logging.error(f"Failed to process BVH file: {e}")


In [7]:
output = 'miscs/h36m_cxk.bvh'
h36m_skel = h36m_skeleton.H36mSkeleton()
_ = h36m_skel.poses2bvh(pose3d_world, output_file=output)

In [2]:
#@markdown We implemented some functions to visualize the face landmark detection results. <br/> Run the following cell to activate the functions.
import cv2
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
import numpy as np
import matplotlib.pyplot as plt
from mediapipe import solutions
from mediapipe.framework.formats import landmark_pb2

# Initialize MediaPipe FaceMesh for video
mp_face_mesh = mp.solutions.face_mesh
mp_drawing = mp.solutions.drawing_utils


def plot_face_blendshapes_bar_graph(face_blendshapes):
  # Extract the face blendshapes category names and scores.
  face_blendshapes_names = [face_blendshapes_category.category_name for face_blendshapes_category in face_blendshapes]
  face_blendshapes_scores = [face_blendshapes_category.score for face_blendshapes_category in face_blendshapes]
  # The blendshapes are ordered in decreasing score value.
  face_blendshapes_ranks = range(len(face_blendshapes_names))

  fig, ax = plt.subplots(figsize=(12, 12))
  bar = ax.barh(face_blendshapes_ranks, face_blendshapes_scores, label=[str(x) for x in face_blendshapes_ranks])
  ax.set_yticks(face_blendshapes_ranks, face_blendshapes_names)
  ax.invert_yaxis()

  # Label each bar with values
  for score, patch in zip(face_blendshapes_scores, bar.patches):
    plt.text(patch.get_x() + patch.get_width(), patch.get_y(), f"{score:.4f}", va="top")

  ax.set_xlabel('Score')
  ax.set_title("Face Blendshapes")
  plt.tight_layout()
  plt.show()

In [4]:
import json
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
import cv2

# Initialize FaceLandmarker
base_options = python.BaseOptions(model_asset_path='face_landmarker_v2_with_blendshapes.task')
options = vision.FaceLandmarkerOptions(
    base_options=base_options,
    output_face_blendshapes=True,
    output_facial_transformation_matrixes=True,
    num_faces=1,
)
detector = vision.FaceLandmarker.create_from_options(options)

# Process video
cap = cv2.VideoCapture('face.mp4')
frame_count = 0
blendshapes_data = []

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Convert frame to RGB
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Detect landmarks
    image = mp.Image(image_format=mp.ImageFormat.SRGB, data=rgb_frame)
    detection_result = detector.detect(image)

    if detection_result.face_blendshapes:
        # Extract face_blendshapes for the first face
        blendshapes = {
            category.category_name: category.score
            for category in detection_result.face_blendshapes[0]
        }
        blendshapes_data.append(blendshapes)
        print(f"Frame {frame_count}: {blendshapes}")

    frame_count += 1

cap.release()

# Save the blendshape data to a JSON file
with open("face_blendshapes.json", "w") as json_file:
    json.dump(blendshapes_data, json_file)

print("Blendshape extraction complete. Saved to face_blendshapes.json")


Frame 0: {'_neutral': 2.503131781850243e-06, 'browDownLeft': 0.003413404570892453, 'browDownRight': 0.0023795110173523426, 'browInnerUp': 0.43352439999580383, 'browOuterUpLeft': 0.20448510348796844, 'browOuterUpRight': 0.106257364153862, 'cheekPuff': 0.0001847489911597222, 'cheekSquintLeft': 1.1876852568093454e-06, 'cheekSquintRight': 1.4340678262669826e-06, 'eyeBlinkLeft': 0.011558977887034416, 'eyeBlinkRight': 0.011141480877995491, 'eyeLookDownLeft': 0.08402687311172485, 'eyeLookDownRight': 0.09577243030071259, 'eyeLookInLeft': 0.0028457948938012123, 'eyeLookInRight': 0.5245062112808228, 'eyeLookOutLeft': 0.489596962928772, 'eyeLookOutRight': 0.004432687070220709, 'eyeLookUpLeft': 0.09839621186256409, 'eyeLookUpRight': 0.08300226181745529, 'eyeSquintLeft': 0.07004984468221664, 'eyeSquintRight': 0.04472121596336365, 'eyeWideLeft': 0.05776858702301979, 'eyeWideRight': 0.04366421326994896, 'jawForward': 0.0004092187446076423, 'jawLeft': 0.0033884155564010143, 'jawOpen': 0.59619659185409

In [11]:
import os
import subprocess
import logging

logging.basicConfig(level=logging.DEBUG)

def process_blendshapes_in_blender(
    blendshapes_file,
    blender_executable=r"C:/Program Files/Blender Foundation/Blender 3.5/blender.exe"
):
    """
    Opens Blender, creates a face rig, and applies blendshape animations.

    Parameters:
        blendshapes_file (str): Path to the JSON file containing blendshape data.
        blender_executable (str): Path to the Blender executable.
    """
    # Check if the blendshapes file exists
    if not os.path.exists(blendshapes_file):
        raise FileNotFoundError(f"The blendshapes file '{blendshapes_file}' does not exist.")
    
    # Create a temporary Python script to run in Blender
    blender_script = f"""
import bpy
import json
import os

# Path to the JSON file with blendshape data
blendshapes_file = r"{os.path.abspath(blendshapes_file)}"

# Load blendshape data
with open(blendshapes_file, "r") as json_file:
    blendshapes_data = json.load(json_file)

# Add the Human Meta-Rig
bpy.ops.object.armature_human_metarig_add()
meta_rig = bpy.context.object

# Generate the Rig
bpy.ops.object.mode_set(mode='OBJECT')
bpy.ops.pose.rigify_generate()
rig = bpy.data.objects["rig"]  # Generated Rigify armature

# Full blendshape-to-bone mapping
blendshape_to_bone = {{
    "_neutral": None,  # No specific control for neutral
    "browDownLeft": "brow.L",
    "browDownRight": "brow.R",
    "browInnerUp": "brow_inner",
    "browOuterUpLeft": "brow_outer.L",
    "browOuterUpRight": "brow_outer.R",
    "cheekPuff": "cheek",
    "cheekSquintLeft": "cheek.L",
    "cheekSquintRight": "cheek.R",
    "eyeBlinkLeft": "eye.L",
    "eyeBlinkRight": "eye.R",
    "eyeLookDownLeft": "eye.L",
    "eyeLookDownRight": "eye.R",
    "eyeLookInLeft": "eye_inner.L",
    "eyeLookInRight": "eye_inner.R",
    "eyeLookOutLeft": "eye_outer.L",
    "eyeLookOutRight": "eye_outer.R",
    "eyeLookUpLeft": "eye.L",
    "eyeLookUpRight": "eye.R",
    "eyeSquintLeft": "eye_squint.L",
    "eyeSquintRight": "eye_squint.R",
    "eyeWideLeft": "eye_wide.L",
    "eyeWideRight": "eye_wide.R",
    "jawForward": "jaw",
    "jawLeft": "jaw.L",
    "jawOpen": "jaw",
    "jawRight": "jaw.R",
    "mouthClose": "mouth_close",
    "mouthDimpleLeft": "mouth_dimple.L",
    "mouthDimpleRight": "mouth_dimple.R",
    "mouthFrownLeft": "mouth_frown.L",
    "mouthFrownRight": "mouth_frown.R",
    "mouthFunnel": "mouth_funnel",
    "mouthLeft": "mouth.L",
    "mouthLowerDownLeft": "mouth_lower.L",
    "mouthLowerDownRight": "mouth_lower.R",
    "mouthPressLeft": "mouth_press.L",
    "mouthPressRight": "mouth_press.R",
    "mouthPucker": "mouth_pucker",
    "mouthRight": "mouth.R",
    "mouthRollLower": "mouth_roll_lower",
    "mouthRollUpper": "mouth_roll_upper",
    "mouthShrugLower": "mouth_shrug_lower",
    "mouthShrugUpper": "mouth_shrug_upper",
    "mouthSmileLeft": "mouth_smile.L",
    "mouthSmileRight": "mouth_smile.R",
    "mouthStretchLeft": "mouth_stretch.L",
    "mouthStretchRight": "mouth_stretch.R",
    "mouthUpperUpLeft": "mouth_upper.L",
    "mouthUpperUpRight": "mouth_upper.R",
    "noseSneerLeft": "nose_sneer.L",
    "noseSneerRight": "nose_sneer.R"
}}

# Set frame range
num_frames = len(blendshapes_data)
bpy.context.scene.frame_start = 1
bpy.context.scene.frame_end = num_frames

# Apply blendshape data to rig bones
for frame, blendshapes in enumerate(blendshapes_data, start=1):
    bpy.context.scene.frame_set(frame)
    for blendshape, value in blendshapes.items():
        bone_name = blendshape_to_bone.get(blendshape)
        if bone_name and bone_name in rig.pose.bones:
            bone = rig.pose.bones[bone_name]
            bone.rotation_euler[0] = value * 3.14  # Example: apply as rotation (adjust scaling)
            bone.keyframe_insert(data_path="rotation_euler", index=0)

# Save the file
output_blend = os.path.splitext(blendshapes_file)[0] + "_rig_with_animation.blend"
bpy.ops.wm.save_as_mainfile(filepath=output_blend)

print("Blendshape animations applied to rig and saved as", output_blend)
    """
    
    # Write the Blender script to a temporary file
    script_file = "apply_blendshapes_to_rig.py"
    with open(script_file, "w") as file:
        file.write(blender_script)
    
    # Run Blender with the temporary script
    try:
        subprocess.run([blender_executable, "--background", "--python", script_file], check=True)
    finally:
        # Clean up the temporary script
        if os.path.exists(script_file):
            os.remove(script_file)

# Example usage
blendshapes_file = r"C:/Users/dodom/OneDrive - Misr International University/Desktop/College/college/Graduation Project/video2bvh-master/video2bvh-master/face_blendshapes.json"  # Replace with the correct path to your blendshapes JSON file
try:
    process_blendshapes_in_blender(blendshapes_file)
except Exception as e:
    logging.error(f"Failed to process blendshapes: {e}")


In [ ]:
# At this point, `keypoints` contains the 3D coordinates (x, y, z) of the face landmarks for each frame
print("Captured Keypoints:")
print(keypoints)  # This will display all stored keypoints across frames